# Week 3 - Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

Following the instructions on coursera page.

In [25]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [26]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
url_data = requests.get(url).text
wiki_data = BeautifulSoup(url_data, 'lxml')

First step is convert the HTML content on **wiki_data** to a dataframe table

In [27]:
column_names = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_names)

content = wiki_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    toronto = toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

In [28]:
print(toronto)

    Postalcode         Borough  \
0            0               0   
1        M1A\n  Not assigned\n   
2        M2A\n  Not assigned\n   
3        M3A\n    North York\n   
4        M4A\n    North York\n   
..         ...             ...   
176      M5Z\n  Not assigned\n   
177      M6Z\n  Not assigned\n   
178      M7Z\n  Not assigned\n   
179      M8Z\n     Etobicoke\n   
180      M9Z\n  Not assigned\n   

                                          Neighborhood  
0                                                    0  
1                                         Not assigned  
2                                         Not assigned  
3                                            Parkwoods  
4                                     Victoria Village  
..                                                 ...  
176                                       Not assigned  
177                                       Not assigned  
178                                       Not assigned  
179  Mimico NW, The Q

Note that the dataframe contains some N/A data. We need to clean this

In [29]:
toronto = toronto[toronto.Borough!='Not assigned\n']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned\n':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1

In [30]:
df = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

In [33]:
for i in range(0, toronto.shape[0]):
    df["Postalcode"][i] = df["Postalcode"][i].rstrip("\n")
    df["Borough"][i] = df["Borough"][i].rstrip("\n")

In [34]:
df

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


Statiscal Data from this dataframe

In [35]:
df.describe()

,Postalcode,Borough,Neighborhood
count,103,103,103
unique,103,10,99
top,M6A,North York,Downsview
freq,1,24,4


Apply neighborhood filter


In [36]:
def filter_neigh(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['Postalcode', 'Borough'])
df_2 = grp.apply(filter_neigh).reset_index(name='Neighborhood')

In [37]:
df_2.describe()
print(df_2.shape)
df_2.head()
# save in csv 
df_2.to_csv('toronto.csv', index=False)

(103, 3)
